In [2]:
import requests
import pandas as pd
from datetime import datetime


ImportError: cannot import name 'Mapping' from 'collections' (C:\Users\mediolanum\AppData\Local\anaconda3\lib\collections\__init__.py)

In [1]:

# downloading news from CNBC web site

# key word that news should include
querry_word = "apple"

params = {
    "queryly_key": "31a35d40a9a64ab3",
    "query": querry_word,
    "endindex": "0",
    "batchsize": "100",
    "callback": "",
    "showfaceted": "true",
    "timezoneoffset": "-120",
    "facetedfields": "formats",
    "facetedkey": "formats|",
    "facetedvalue":
    "!Press Release|",
    "needtoptickers": "1",
    "additionalindexes": "4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"
}

# parts of news article that will be included, in our project we use only Title and Publication Date but Description is also 
# present as we can train model on description of articles
goal = ["cn:title", "_pubDate", "description"]


def main(url):
    with requests.Session() as req:
        allin = []
        for page, item in enumerate(range(5000, 20000, 100)):  # to get more or less news change range
                                                          # later news in the begging and old news are in the end
            print(f"Extracting Page# {page +1}")
            params["endindex"] = item
            r = req.get(url, params=params).json()
            for loop in r['results']:
                allin.append([loop[x] for x in goal])
        new = pd.DataFrame(
            allin, columns=["Title", "date", "Description"])
        new.to_csv("/datasets/data.csv", index=False)


main("https://api.queryly.com/cnbc/json.aspx")

NameError: name 'requests' is not defined

In [ ]:

# combining the news articles data and AAPL stock's data 

# AAPL.csv is downloaded from Kaggle dataset about the Apple's stock prices 
a = pd.read_csv("AAPL.csv")
b = pd.read_csv("data.csv")


# since data from CNBC and Kaggle dataset have different format for "date" (day-month-year:00:00 and day/month/year)
# we change their format and standartize it
a['date'] = pd.to_datetime(a['date'], dayfirst=True, utc = False).dt.tz_localize(None).dt.strftime("%Y-%m-%d")
b['date'] = pd.to_datetime(b['date'], dayfirst=True, utc = False).dt.tz_localize(None).dt.strftime("%Y-%m-%d")


# saving cvs files 
a.to_csv("a.csv")
b.to_csv("b.csv")

In [ ]:

# merging 2 cvs files, saving only entries that share same "date" value so we have infomation about stock price and 
# news articles that were published that day regarfing the company
merged_df = pd.merge(a, b, on='date')
print(a.info(), b.info())

# saving merged dataset as "output.csv"
merged_df.to_csv("output.csv")